# Restricted Boltzmann Machines (RBMs)
Let us assume we have some random samples of fixed-length binary sequences that we wish to represent in an RBM.

In [ ]:
import numpy as np
data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,1,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])

### Constructing the model

The RBM is an energy-based model defined by a quadratic weight matrix $W$ and linear biases $b$ and $c$.  The energy $E(v, h)$ is a function of the binary visible units $v$ and the hidden units $h$.

$-E(v, h)=v^T W h + c^T h + b^T v$

Our RBM has 6 visible, and say 2 hidden units.

In [ ]:
num_visible = 6 # = data.shape[0]
num_hidden  = 2

In [ ]:
weights = np.random.randn(num_visible, num_hidden)
weights

We append the biases to the weight matrix, and later add respective terms to $h$ and $v$.

First, we append $c^T h$, which addes 2 parameters to $W$.

In [ ]:
weights = np.insert(weights, 0, 0, axis=0)
weights

Next we add $b^T v$, adding 6 more parameters.

In [ ]:
weights = np.insert(weights, 0, 0, axis=1)
weights

In [ ]:
weights.shape

### Training the model

In [ ]:
nb_epoch = 100
learning_rate = 0.1
num_examples = data.shape[0]

Each training data is given as a row in `data`.  We append an extra column for the bias term $b^T v$.

In [ ]:
training_data = np.insert(data, 0, 1, axis=1)
training_data

In [ ]:
training_data.shape

To train the model, we need to compute the hidden activations, probabilities, and states.

- Activation: $a(v)=v^T W + b$
- Probability: $p(h=1|v)=1/(1+\exp{(-a(v))})$
- State: $h = \langle h \rangle$.  The state is obtained by Gibbs sampling.

In [ ]:
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def compute_hidden(visible):
    num_examples = visible.shape[0]
    pos_hidden_activations = np.dot(visible, weights)      
    pos_hidden_probs = logistic(pos_hidden_activations)
    pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, num_hidden + 1)
    return pos_hidden_activations, pos_hidden_probs, pos_hidden_states

test_data = training_data[:1]
a, p, hidden_states = compute_hidden(test_data)
print("input data:", test_data)
print("activation:", a)
print("probabilit:", p)
print("hidden sta:", hidden_states)

We also need to compute the visible units to compare with the visible activations, probability.

- visible activation: $a(h) = W h + c^T h$
- visible probability: $P(a=1|h) = 1/(1+\exp{(-a(h))})$

In [ ]:
def compute_visible(hidden_states):
    num_examples = hidden_states.shape[0]
    neg_visible_activations = np.dot(hidden_states, weights.T)
    neg_visible_probs = logistic(neg_visible_activations)
    neg_visible_probs[:, 0] = 1 # Fix the bias unit.
    return neg_visible_probs

neg_visible_probs = compute_visible(hidden_states)
print("negative visible probabilities:", neg_visible_probs)

After computing probabilities for the visible units, we can use these to estimate hidden units again, a process called daydreaming.

In [ ]:
def daydream(neg_visible_probs):
    neg_hidden_activations = np.dot(neg_visible_probs, weights)
    neg_hidden_probs = logistic(neg_hidden_activations)
    return neg_hidden_probs

Training is done in three steps:

- Estimate the hidden states by Gibbs sampling.
- From step 1 compute the probability of the visible units.  Comparison with the training data gives us the error function.
- Update the weights by minimizing the Energy.

Let's take the derivative of $E(v, h)$:

$\partial_{W_{ij}} E = -v^T h$

In [ ]:
error = []
for epoch in range(nb_epoch):
    pos_hidden_act, pos_hidden_prob, pos_hidden_stat = compute_hidden(training_data)
    
    neg_visible_prob = compute_visible(pos_hidden_stat)
    neg_hidden_prob = daydream(neg_visible_prob)
    
    pos_associations = np.dot(training_data.T, pos_hidden_prob)
    neg_associations = np.dot(neg_visible_prob.T, neg_hidden_prob)
    
    # Update the weights:
    weights += learning_rate * ((pos_associations-neg_associations)/float(num_examples))
    
    # Error:
    err = np.sum( (training_data - neg_visible_prob)**2 )
    error.append(err)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(error)
plt.xlabel('epoch')
plt.ylabel('Error')